# Imports
This notebook uses data available to FIREMAN members on [LUT sharepoint](https://lut.sharepoint.com/:f:/r/sites/o365fireman/Shared%20Documents/Colab_PowerElectronicConverter/PEC_datasets/New%20Data?csf=1&web=1&e=JINTw3)

In [1]:
import logging
import sys
import pandas as pd

# to save results to data directory
module_path = '..'
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [2]:
import scipy.io

# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

import dill as pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
logging.basicConfig(format='%(asctime)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger("TimeSeries")
logger.setLevel(logging.INFO)
# temporarily remove deprecation warnings
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

output_dir = 'pec_output'

# Dataset

In [3]:
with open(f"_data/PEC/dataset.pkl", "rb") as fh:
    data_df_final = pickle.load(fh)

with open(f"_data/PEC/outlier_model.pkl", "rb") as fh:
    outlier_model = pickle.load(fh)


## Simple display of the dataset

In [4]:
data_df_final.head()

,f_c,P,m_d,m_q,theta,P_ref,V_DC,V_phaseA,V_phaseB,V_phaseC,I_phaseA,I_phaseB,I_phaseC,label,fault,
0,50.0,2499.997221,311.0,0.0,312.486512,2500.0,800.0,2.487576,-270.569073,268.081497,-0.392875,-4.447507,4.840382,LL_Fault,0,0
1,50.0,2499.997250,311.0,0.0,312.502220,2500.0,800.0,7.372088,-272.944264,265.572176,-0.308598,-4.494417,4.803015,LL_Fault,0,1
2,50.0,2499.997280,311.0,0.0,312.517928,2500.0,800.0,12.254782,-275.252112,262.997330,-0.224245,-4.540218,4.764463,LL_Fault,0,2
3,50.0,2499.997310,311.0,0.0,312.533636,2500.0,800.0,17.134452,-277.492044,260.357593,-0.139837,-4.584899,4.724735,LL_Fault,0,3
4,50.0,2499.997341,311.0,0.0,312.549344,2500.0,800.0,22.009894,-279.663510,257.653616,-0.055394,-4.628448,4.683842,LL_Fault,0,4


In [5]:
data_df_final.plot(subplots=True, figsize=(16, 15));

04/22/2022 12:15:33 PM: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [6]:
height = 4
width = 5


def plot_save_mp(outlier_model, file_name, s_index=0, e_index=-1, sci_index=False):
    fig = plt.figure()
    if e_index > 0: # Adjust for full dataset
        s_index_mp = s_index - outlier_model.ts_size
        e_index_mp = e_index - outlier_model.ts_size
        local_range = range(s_index,e_index)
    else:
        s_index_mp = s_index
        e_index_mp = e_index
        local_range = list(range(0, len(outlier_model.max_val) - 1))
    plt.plot(local_range, outlier_model.max_val[s_index_mp:e_index_mp], 'b--', label='max')
    plt.plot(local_range, outlier_model.max_mean[s_index_mp:e_index_mp], 'y--', label=r'$\mu$')
    plt.plot(local_range, outlier_model.max_std_dev[s_index_mp:e_index_mp], 'g--', label=r'$\sigma$')
    if sci_index:
        plt.ticklabel_format(axis="y", style="sci", scilimits=(3, 3))
    plt.xlabel('Time (s)')
    plt.ylabel('Matrix Profile Values')

    first_detect = True
    for i in outlier_model.anomalies:
        if i in local_range:
            if e_index < 0:
                i = i - outlier_model.ts_size # Adjust for full dataset
            if first_detect:
                plt.axvline(x=i, color='r', linestyle='-', label='detect')
                first_detect = False
            else:
                plt.axvline(x=i, color='r', linestyle='-', )

    plt.legend(loc=0)

    plt.show()
    fig.set_size_inches(w=width, h=height)
    plt.savefig(file_name)

In [7]:
def plot_save_freq(data, file_name, s_index=0, e_index=-1, sci_index=False, height=4, width=5):
    fig = plt.figure()
    plt.plot(data[s_index:e_index])
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    if sci_index:
        plt.ticklabel_format(axis="y", style="sci", scilimits=(3, 3))


    plt.show()
    fig.set_size_inches(w=width, h=height)
    plt.savefig(file_name)

In [8]:
#All results
outlier_key = "f_c"
data_f_c = data_df_final[outlier_key]

In [9]:
height = 4
width = 6
plot_save_freq(data_f_c, f'{output_dir}/base_sig_{outlier_key}_all.pgf', sci_index=True, height=height, width=width)

/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2592244377.py:10: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [10]:
fig = plt.figure()
# local_range = list(range(0, len(outlier_model.max_val) - 1))
plot_save_mp(outlier_model, f'{output_dir}/mp_hist_{outlier_key}.pgf', sci_index=True)

/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2610879682.py:36: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [11]:
height = 4
width = 5

s_index = 40200
e_index = 41200

plot_save_freq(data_f_c, f'{output_dir}/base_sig_{outlier_key}_f1.pgf', s_index=s_index, e_index=e_index,
               sci_index=True, height=height, width=width)
s_index = 40200
e_index = 46000

plot_save_mp(outlier_model, f'{output_dir}/mp_hist_{outlier_key}_f1.pgf', s_index=s_index, e_index=e_index,
             sci_index=True)


/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2592244377.py:10: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2610879682.py:36: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [12]:
s_index = 115000
e_index = 165000

plot_save_freq(data_f_c, f'{output_dir}/base_sig_{outlier_key}_f2.pgf', s_index=s_index, e_index=e_index,
            sci_index=False, height=height, width=width)

plot_save_mp(outlier_model, f'{output_dir}/mp_hist_{outlier_key}_f2.pgf', s_index=s_index, e_index=e_index,
             sci_index=False)

/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2592244377.py:10: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2610879682.py:36: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [13]:
s_index = 196000
e_index = 245000

plot_save_freq(data_f_c, f'{output_dir}/base_sig_{outlier_key}_f3.pgf', s_index=s_index, e_index=e_index,
            sci_index=False, height=height, width=width)

plot_save_mp(outlier_model, f'{output_dir}/mp_hist_{outlier_key}_f3.pgf', s_index=s_index, e_index=e_index,
             sci_index=False)


/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2592244377.py:10: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2610879682.py:36: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [14]:
s_index = 280000
e_index = 285500
plot_save_freq(data_f_c, f'{output_dir}/base_sig_{outlier_key}_f4.pgf', s_index=s_index, e_index=e_index,
            sci_index=True, height=height, width=width)

plot_save_mp(outlier_model, f'{output_dir}/mp_hist_{outlier_key}_f4.pgf', s_index=s_index, e_index=e_index,
             sci_index=True)


/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2592244377.py:10: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/2610879682.py:36: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [15]:
# adapted from https://stackoverflow.com/questions/44951911/plot-a-binary-timeline-in-matplotlib?answertab=modifieddesc#tab-top
width = 5.5
height = 2.5

#create a time series s with dates as index and 0 and 1 for events
df = data_df_final.copy()
my_column_changes = df["fault"].shift() != df["fault"]

events = df[my_column_changes]

fault_start = events.loc[events["fault"] == 1].index.tolist()
fault_end = events.loc[events["fault"] == 0].index.tolist()

no_fault_no_ends = fault_end [1:]

times_faults = list(zip(fault_start, no_fault_no_ends))
fault_start.append(df.index[-1]) # needs last element to finalize graph
times_no_faults = list(zip(fault_end ,fault_start))

bar_green =  list(map(lambda x: (x[0], x[1] - x[0]) , times_no_faults))
bar_red = list(map(lambda x: (x[0], x[1] - x[0]) , times_faults))

fig, ax = plt.subplots(figsize=(width, height))

plt.broken_barh(bar_green, (-1, 1), color="lightgreen")
plt.broken_barh(bar_red, (-1, 1), color="lightsalmon")

ax.vlines(
    outlier_model.anomalies,
    ymin=-1,
    ymax=0,
    linewidth=2,
    colors='r',
    linestyle='--', label='detection')

ax.legend()
#format axes
ax.margins(0)
ax.set_yticks([])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Ground Truth')
plt.tight_layout()
plt.show()

fig.set_size_inches(w=width, h=height)
plt.savefig(f'{output_dir}/outlier_result_{outlier_key}.pgf')

/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/1257285567.py:43: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [16]:
#create a time series s with dates as index and 0 and 1 for events
#create a time series s with dates as index and 0 and 1 for events
dates = range(1, 101)
events = np.random.random_integers(0, 1, size=len(dates))
s = pd.Series(events, index=dates)

fig, ax = plt.subplots(figsize=(6, 2))

# plot green for event==1
s1 = s[s == 1]
# inxval = matplotlib.dates.date2num(s1.index.to_pydatetime())
times1 = list(zip(s1.index, np.ones(len(s1))))
test = list(s1)
plt.broken_barh(times1, (-1, 1), color="lightgreen")
# plot red for event==0
s2 = s[s == 0]
# inxval = matplotlib.dates.date2num(s2.index.to_pydatetime())

times2 = list(zip(s2.index, np.ones(len(s2))))

plt.broken_barh(times2, (-1, 1), color="lightsalmon")
ax.vlines(
    [5, 50, 100],
    ymin=-1,
    ymax=0,
    linewidth=2,
    colors='firebrick',
    linestyle='--', label='detection')

ax.legend()
#format axes
ax.margins(0)
ax.set_yticks([])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Anomaly')
plt.tight_layout()
plt.show()

/var/folders/rt/82pssgq96l9d_s3skbv32z5m0000gn/T/ipykernel_26884/3832441231.py:37: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
